# Text Baseline model testing Conv fusion (OverSampler, cosine scheduling, BCE loss):

In [29]:
import sys
root = '../../'
sys.path.append(root)   # Done to be able to import the packages and functions

import Utils.hico_evaluation.evaluation as ev
from Utils.custom_sampler import OverSampler
from Utils.custom_loss import MaskedBCELoss
from Utils.annotation_preprocessing import _load_csv_to_tensor
from Utils.train_val_split import train_val_split_hico
from hoi_classifiers import ConvolutionFusionModel

import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

import random

In [30]:
seed = 42   #note that the model parameters will still be randomly initiated
torch.manual_seed(seed)
random.seed(seed)

### Loading the training set:


In [31]:
# Loading the data:
X_train = torch.load(root + "Embeddings/Text_Embeddings/train.pt")
y_train = _load_csv_to_tensor(root + "anno/added/anno_augmented_train.csv").T # Transpose to make both first dimensions the #samples.
y_train[y_train.isnan()] = -1

In [32]:
X_test = torch.load(root + "Embeddings/Text_Embeddings/test.pt")
y_test = _load_csv_to_tensor(root + "anno/added/anno_augmented_test.csv").T # Transpose to make both first dimensions the #samples.
y_test[y_test.isnan()] = -1

#### Training Preparations:

In [33]:
train_dataset = TensorDataset(X_train, y_train)

In [34]:
# Batch size:
bs = 512

sampler = OverSampler(y_train[:,:600], shuffle=True)

train_dataloader = DataLoader(
    train_dataset, batch_size=bs, num_workers=4, sampler=sampler)


In [35]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [36]:
nr_of_kernels = 1
model = ConvolutionFusionModel(512,4, nr_of_kernels, 655, 797)
model = model.to(device)

In [37]:
classweights = torch.cat((torch.ones(600),torch.ones(197)*0.5)).to(device)      # The hoi classes weigh twice as much as the seperate classes

criterion = MaskedBCELoss(ignore_label=0, convert_target_to_01= True, weight=classweights)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1)

#### Training loop:

In [38]:
num_epochs = 67
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        running_loss += loss.item()

    scheduler.step()


    print(f"Epoch {epoch+1:0{len(str(num_epochs))}}/{num_epochs}, Loss: {running_loss/len(train_dataloader):.12f}")

Epoch 01/67, Loss: 0.084648016615
Epoch 02/67, Loss: 0.022084008370
Epoch 03/67, Loss: 0.021906215145
Epoch 04/67, Loss: 0.021625630025
Epoch 05/67, Loss: 0.021441778925
Epoch 06/67, Loss: 0.020869138477
Epoch 07/67, Loss: 0.019155249491
Epoch 08/67, Loss: 0.016812906877
Epoch 09/67, Loss: 0.015376122699
Epoch 10/67, Loss: 0.014833144096
Epoch 11/67, Loss: 0.013734466110
Epoch 12/67, Loss: 0.011789498022
Epoch 13/67, Loss: 0.010502855007
Epoch 14/67, Loss: 0.009820035475
Epoch 15/67, Loss: 0.009543246211
Epoch 16/67, Loss: 0.009172485292
Epoch 17/67, Loss: 0.008474406433
Epoch 18/67, Loss: 0.008004839496
Epoch 19/67, Loss: 0.007739003390
Epoch 20/67, Loss: 0.007616725290
Epoch 21/67, Loss: 0.007535798864
Epoch 22/67, Loss: 0.007245238430
Epoch 23/67, Loss: 0.007009291689
Epoch 24/67, Loss: 0.006872433862
Epoch 25/67, Loss: 0.006787624788
Epoch 26/67, Loss: 0.006777015719
Epoch 27/67, Loss: 0.006588596477
Epoch 28/67, Loss: 0.006436969502
Epoch 29/67, Loss: 0.006317820999
Epoch 30/67, L

### Checking the filters of the trained model:

In [39]:
list(model.conv1.parameters())

[Parameter containing:
 tensor([[[0.5828],
          [0.6298],
          [0.6385],
          [0.9296]]], device='cuda:0', requires_grad=True),
 Parameter containing:
 tensor([-0.2179], device='cuda:0', requires_grad=True)]

#### Evaluating model:

In [40]:
test_pred = model.forward(X_test.to(device))
test_map = ev.eval_vo(test_pred[:,:600].T.cpu().detach().numpy(), y_test[:,:600].T.numpy(),600)[0].mean()

In [41]:
print("mAP: ", test_map * 100)

mAP:  52.03413253116235


##### Fewshot evaluation:

In [42]:
one_shot_list = torch.where((0 < (y_train[:,:600] == 1).sum(dim=0)) & ((y_train[:,:600] == 1).sum(dim=0) <= 1))[0]
few_shot_list_5 = torch.where((0 < (y_train[:,:600] == 1).sum(dim=0)) & ((y_train[:,:600] == 1).sum(dim=0) <= 5))[0]
few_shot_list_10 = torch.where((0 < (y_train[:,:600] == 1).sum(dim=0)) & ((y_train[:,:600] == 1).sum(dim=0) <= 10))[0]

one_shot_map = ev.eval_vo(test_pred[:,one_shot_list].T.cpu().detach().numpy(), y_test[:,one_shot_list].T.numpy(),len(one_shot_list))[0].mean()
few_5_shot_map = ev.eval_vo(test_pred[:,few_shot_list_5].T.cpu().detach().numpy(), y_test[:,few_shot_list_5].T.numpy(),len(few_shot_list_5))[0].mean()
few_10_shot_map = ev.eval_vo(test_pred[:,few_shot_list_10].T.cpu().detach().numpy(), y_test[:,few_shot_list_10].T.numpy(),len(few_shot_list_10))[0].mean()

print("Few@1 mAP: ", one_shot_map * 100)
print("Few@5 mAP: ", few_5_shot_map * 100)
print("Few@10 mAp: ", few_10_shot_map * 100)

Few@1 mAP:  30.875684056566772
Few@5 mAP:  37.38865785417163
Few@10 mAp:  38.55030103353095
